# Data Cleaning

In [1]:
import pandas as pd

In [2]:
#import raw dataset
df = pd.read_csv('./data/Video_Games_Sales_as_at_22_Dec_2016.csv')

## Initial Data Brushup

In [3]:
# Clean up column names
df.columns = df.columns.str.lower()
df.rename(columns={'year_of_release': 'release_year'}, inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16719 entries, 0 to 16718
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          16717 non-null  object 
 1   platform      16719 non-null  object 
 2   release_year  16450 non-null  float64
 3   genre         16717 non-null  object 
 4   publisher     16665 non-null  object 
 5   na_sales      16719 non-null  float64
 6   eu_sales      16719 non-null  float64
 7   jp_sales      16719 non-null  float64
 8   other_sales   16719 non-null  float64
 9   global_sales  16719 non-null  float64
 10  critic_score  8137 non-null   float64
 11  critic_count  8137 non-null   float64
 12  user_score    10015 non-null  object 
 13  user_count    7590 non-null   float64
 14  developer     10096 non-null  object 
 15  rating        9950 non-null   object 
dtypes: float64(9), object(7)
memory usage: 2.0+ MB


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16719 entries, 0 to 16718
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          16717 non-null  object 
 1   platform      16719 non-null  object 
 2   release_year  16450 non-null  float64
 3   genre         16717 non-null  object 
 4   publisher     16665 non-null  object 
 5   na_sales      16719 non-null  float64
 6   eu_sales      16719 non-null  float64
 7   jp_sales      16719 non-null  float64
 8   other_sales   16719 non-null  float64
 9   global_sales  16719 non-null  float64
 10  critic_score  8137 non-null   float64
 11  critic_count  8137 non-null   float64
 12  user_score    10015 non-null  object 
 13  user_count    7590 non-null   float64
 14  developer     10096 non-null  object 
 15  rating        9950 non-null   object 
dtypes: float64(9), object(7)
memory usage: 2.0+ MB


In [5]:
df.head(3)

,name,platform,release_year,genre,publisher,na_sales,eu_sales,jp_sales,other_sales,global_sales,critic_score,critic_count,user_score,user_count,developer,rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E


In [6]:
# Convert release_year from float to datetime
# Source 1
df['release_year'] = pd.to_datetime(df.release_year, format='%Y')

### Should the floats be ints?

For some reason, pandas imported all numeric columns as floats. Let's check to see if this was necessary in columns that should be ints

columns to check:
- user count
- critic count
- critic score

In [7]:
# Check if anything would be lost if user count was converted to int
print((df.user_count.value_counts() % 1).value_counts())

# Check if anything would be lost if critic count was converted to int
print((df.critic_count.value_counts() % 1).value_counts())

# Check if anything would be lost if critic score was converted to int
print((df.critic_score.value_counts() % 1).value_counts())

0    888
Name: user_count, dtype: int64
0    106
Name: critic_count, dtype: int64
0    82
Name: critic_score, dtype: int64


Determined that all three columns should be converted to int.

Source 2 informed me that pandas most likely converted the columns to floats because they contain null values. And ints don't like that, apparently

In [8]:
# Source 2
df.user_count = df.user_count.astype('Int64')
df.critic_count = df.critic_count.astype('Int64')
df.critic_score = df.critic_score.astype('Int64')

### Investigate why user_score is an object instead of a numeric

In [9]:
df.user_score.value_counts()

tbd    2425
7.8     324
8       290
8.2     282
8.3     254
       ... 
9.6       2
1         2
1.5       2
9.7       1
0         1
Name: user_score, Length: 96, dtype: int64

In [10]:
# Contrast regular value counts with the function that forces 
# non-numerics to nulls
# Source 4
pd.to_numeric(df.user_score, errors='coerce').value_counts()

7.8    324
8.0    290
8.2    282
8.3    254
8.5    253
      ... 
9.6      2
0.6      2
1.9      2
9.7      1
0.0      1
Name: user_score, Length: 95, dtype: int64

> Determine what to do with 'tbd'

'tbd' stands for 'to be determined'. In other words, 'tbd' is a placeholder for later. We only have the data as it currently exists. Therefore, 'tbd' is effectively the same as 'null'.

> Conclusion: 

Replace 'tbd' with 'null' in user_score

In [11]:
df.user_score = pd.to_numeric(df.user_score, errors='coerce')

## Save cleaned data into new datafile

What about nulls?
Decide to leave the nulls for now. They can be easily dropped if necessary when performing eda

In [12]:
df.to_csv('./data/cleaned_video_game_sales.csv', index=False)

## Sources

1. https://stackoverflow.com/questions/46658232/pandas-convert-column-with-year-integer-to-datetime
2. https://stackoverflow.com/questions/21287624/convert-pandas-column-containing-nans-to-dtype-int
3. https://stackoverflow.com/questions/21771133/finding-non-numeric-rows-in-dataframe-in-pandas
4. https://stackoverflow.com/questions/34794067/how-to-set-a-cell-to-nan-in-a-pandas-dataframe